In [1]:
from dataclasses import dataclass, field
import pandas as pd
import numpy as np
import pickle
import json
import matplotlib.pyplot as plt
%matplotlib inline
from data_science import PrepTools, ModelSelectionTools
from gauss_rank_scaler import GaussRankScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer, QuantileTransformer, RobustScaler, StandardScaler
from xgboost import DMatrix, XGBClassifier
from xgboost import train as Xtrain
from xgboost import cv as Xcv
from xgboost.callback import reset_learning_rate
from bankroll_calc import BankrollCalculator
from recursive_selection import FeatureSelector
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

### Outside of a few limited success, the "big" (I know it's not that big, but as far as baseball goes it's big so shut up). data approach has failed. Like I said, there have been some small sucesses here and there, but for the most part things are not working out. The overarching trends I was hoping to find in the data throughout the history of baseball don't seem to exist. The reason being, I assume, is that there is too much variance team to team season by season and thus one season cannot accurately predict the next. So here is a last ditch, hail mary approach I've been kicking around: let's go small. Create a workflow on a single season that retrains DAILY... We are going to have reduce the feature set dramatically, but I think that's possible. My thought here is simple: can we capture the flow of a season? Can we find a team that Vegas undervalued? Let's find out. 

# Ideas: 

* Try using ONLY 2019 season- Grade: Fail - 52% accuracy, money lost
* Try using 2010-2018 seasons to train model, retrain daily / weekly / ... for 2019

In [2]:
df = pd.read_csv("./adv_metrics/current.csv.gz", compression = "gzip")

with open("./adv_metrics/adv_metrics_drop_cols.json", "r+") as f:
    
    remove = json.load(f)
    
df.head()

,date,home_close_TJ,home_close_ou,home_closing,home_open_TJ,home_open_ou,home_opening,home_runline,home_runline_juice,is_doubleheader,...,home_H_streak,home_record,run_differential_hm,avg_margin_hm,distance_traveled,road_streak,road_R_streak,road_record,run_differential_rd,avg_margin_rd
0,2010-04-04,-115,9.0,-117.0,-104,9.0,-114.0,NaN,NaN,0,...,0.0,0.0,0.0,0.0,289.0,0.0,0.0,0.0,0.0,0.0
1,2010-04-05,105,7.5,175.0,-120,7.5,175.0,NaN,NaN,0,...,0.0,0.0,0.0,0.0,195.0,0.0,0.0,0.0,0.0,0.0
2,2010-04-05,-105,7.0,137.0,-105,7.0,110.0,NaN,NaN,0,...,0.0,0.0,0.0,0.0,1095.0,0.0,0.0,0.0,0.0,0.0
3,2010-04-05,110,8.5,-128.0,100,8.5,-135.0,NaN,NaN,0,...,0.0,0.0,0.0,0.0,2441.0,0.0,0.0,0.0,0.0,0.0
4,2010-04-05,-115,7.0,-122.0,-115,7.0,-115.0,NaN,NaN,0,...,0.0,0.0,0.0,0.0,1030.0,0.0,0.0,0.0,0.0,0.0


## PROOF OF CONCEPT: SMALL DATA APPROACH :( 

In [3]:
working = df[df.season < 2019].reset_index(drop = True)

working.date = pd.to_datetime(working.date, format = "%Y-%m-%d")

In [4]:
d = {}

counter = 1

for day in working.date.unique():
    
    d[day] = counter
    
    counter += 1
    
working["day_num"] = working.date.map(d)

working = working.drop("date", axis = 1)

In [4]:
pd.set_option("max.columns", 200)

working = working.drop(columns = ["home_close_TJ", "home_close_ou", "home_closing", "home_open_TJ", 
                                 "home_open_ou", "home_opening", "home_runline", "home_runline_juice",
                                 "is_doubleheader", "road_close_TJ", "road_close_ou", 
                                 "road_closing", "road_open_TJ", "road_open_ou", "road_opening",
                                 "road_runline", "road_runline_juice", "is_tripleheader", "season",
                                 "home_team", "road_team", "home_starter", "road_starter", "elo1_pre",
                                  "elo2_pre", "rating_prob1", "score1", "score2", "home_favorite"])

In [5]:
X = working.drop(columns = ["home_win", "date"])

y = working.home_win

X = PrepTools(X, y).scale_features()

In [6]:
l1_params = {
    
    "penalty" : "l1",
    
    "solver" : "liblinear",
    
    "C" : 0.1,
    
    "fit_intercept" : False
    
}

selector_params = {
    
    "X" : X,
    
    "y" : y,
    
    "algorithm" : LogisticRegression(),
    
    "params" : l1_params
    
    
}

selector = FeatureSelector(**selector_params)

In [247]:
selector.recursive_selection()

'Cannot reduce feature frame anymore. Reduce drop size if desired'

In [248]:
l1_subset, l1_eval = list(selector.best_subset), selector.best_eval

In [250]:
l1_subset, l1_eval

(['road_starter_season_wRAA', 'road_payout', 'line_movement'],
 0.562309238601301)

In [251]:
l2_params = {
    
    "penalty" : "l2",
    
    "solver" : "lbfgs",
    
    "C" : 0.1,
    
    "fit_intercept" : False,
    
    "max_iter" : 1000
    
}

selector_params = {
    
    "X" : X,
    
    "y" : y,
    
    "algorithm" : LogisticRegression(),
    
    "params" : l2_params
    
    
}

selector = FeatureSelector(**selector_params)

In [252]:
selector.recursive_selection()

'Cannot reduce feature frame anymore. Reduce drop size if desired'

In [253]:
l2_subset, l2_eval = list(selector.best_subset), selector.best_eval

In [254]:
l2_subset, l2_eval

(['road_starter_season_wRAA', 'road_payout', 'line_movement'],
 0.5625876070549574)

In [255]:
xgb_params = {
    
    "max_depth" : 2,
    
    "colsample_by_tree" : 0.5,
    
    "reg_alpha" : 0.,
    
    "reg_lambda" : 0.
    
}

selector_params = {
    
    "X" : X,
    
    "y" : y,
    
    "algorithm" : XGBClassifier(),
    
    "params" : xgb_params
    
    
}

selector = FeatureSelector(**selector_params)

In [256]:
selector.recursive_selection()

'Cannot reduce feature frame anymore. Reduce drop size if desired'

In [257]:
xgb_subset, xgb_eval = list(selector.best_subset), selector.best_eval

In [258]:
xgb_subset, xgb_eval

(['road_relief_FIP', 'home_payout', 'road_payout'], 0.5752533124941807)

In [259]:
working.head()

,date,home_wOBA,home_wRAA,home_wRC,home_OPS,home_relief_wOBA,home_relief_wRAA,home_relief_wRC,home_relief_FIP,home_relief_WHIP,home_relief_ERA,home_relief_K_BB,home_relief_K_9,home_starter_career_wOBA,home_starter_career_wRAA,home_starter_career_wRC,home_starter_career_FIP,home_starter_career_WHIP,home_starter_career_ERA,home_starter_careerK/BB,home_starter_careerK/9,home_starter_career_AVGIP,home_starter_season_wOBA,home_starter_season_wRAA,home_starter_season_wRC,home_starter_season_FIP,home_starter_season_WHIP,home_starter_season_ERA,home_starter_seasonK/BB,home_starter_seasonK/9,home_starter_seasonAVG_IP,road_wOBA,road_wRAA,road_wRC,road_OPS,road_relief_wOBA,road_relief_wRAA,road_relief_wRC,road_relief_FIP,road_relief_WHIP,road_relief_ERA,road_relief_K_BB,road_relief_K_9,road_starter_career_wOBA,road_starter_career_wRAA,road_starter_career_wRC,road_starter_career_FIP,road_starter_career_WHIP,road_starter_career_ERA,road_starter_careerK/BB,road_starter_careerK/9,road_starter_career_AVGIP,road_starter_season_wOBA,road_starter_season_wRAA,road_starter_season_wRC,road_starter_season_FIP,road_starter_season_WHIP,road_starter_season_ERA,road_starter_seasonK/BB,road_starter_seasonK/9,road_starter_seasonAVG_IP,home_payout,road_payout,rating1_pre,rating2_pre,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,home_win,line_movement,money_scalar,batting_park_factor,home_streak,home_H_streak,home_record,run_differential_hm,avg_margin_hm,distance_traveled,road_streak,road_R_streak,road_record,run_differential_rd,avg_margin_rd
0,2010-04-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.854701,2.070000,1536.601,1562.673,57.031,61.828,27.349300,46.7039,1,1,-3.0,106.0,0.0,0.0,0.0,0.0,0.0,289.0,0.0,0.0,0.0,0.0,0.0
1,2010-04-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.750000,1.500000,1460.662,1537.875,49.834,62.166,8.394200,49.4957,0,0,0.0,99.0,0.0,0.0,0.0,0.0,0.0,195.0,0.0,0.0,0.0,0.0,0.0
2,2010-04-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.370000,1.657895,1513.910,1500.945,56.701,60.524,7.593298,46.4172,0,-1,27.0,99.0,0.0,0.0,0.0,0.0,0.0,1095.0,0.0,0.0,0.0,0.0,0.0
3,2010-04-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.781250,2.130000,1543.114,1520.771,54.034,55.637,16.337200,26.8276,1,-1,7.0,96.0,0.0,0.0,0.0,0.0,0.0,2441.0,0.0,0.0,0.0,0.0,0.0
4,2010-04-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.819672,2.120000,1478.280,1500.407,63.333,58.357,64.723700,40.3119,0,1,-7.0,99.0,0.0,0.0,0.0,0.0,0.0,1030.0,0.0,0.0,0.0,0.0,0.0


In [6]:
keep_cols = ["home_wRC", "home_relief_wRC", "home_starter_season_wRC", "road_wRC",
            "road_relief_wRC", "road_starter_season_wRC", "home_payout", "road_payout",
            "rating1_pre", "rating2_pre", "pitcher1_rgs", "pitcher2_rgs", "pitcher1_adj",
            "pitcher2_adj", "money_scalar", "batting_park_factor", "home_H_streak", "run_differential_hm",
            "road_R_streak", "run_differential_rd", "home_win"]

In [7]:
train_label = working.home_win

train_weights = working.home_payout - 1

train1 = working[keep_cols].drop("home_win", axis = 1)

train2 = working.drop(columns = ["date", "home_win"])

### Prepare / Tune XGBoost model - start with sklearn API and tune booster parameters, then switch to Python API to train 

In [8]:
from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials
from hyperopt.pyll import scope
from hyperopt.pyll.stochastic import sample
from sklearn.model_selection import cross_val_score

In [327]:
hyperopt_space = {
    
    "n_estimators" : scope.int(hp.quniform("n_estimators", 100, 1000, 100)),
    
    "max_depth" : scope.int(hp.quniform("max_depth", 1, 16, 1)),
    
    "min_child_weight" : scope.int(hp.quniform("min_child_weight", 1, 8, 1)),
    
    "learning_rate" : hp.loguniform("learning_rate", np.log(0.01), np.log(0.50)),
    
    "reg_alpha" : hp.uniform("reg_alpha", 0., 1.),
    
    "reg_lambda" : hp.uniform("reg_lambda", 0., 1.),
    
    "colsample_by_tree" : hp.loguniform("colsample_by_tree", np.log(0.6), np.log(1.0)),
}

def optimize(space, xtrain, ytrain, num_eval):
    
    def objective(params):

        try:

            xgb = XGBClassifier().set_params(**params)

            score = cross_val_score(xgb, xtrain, ytrain, cv = 5,
                                   scoring = "roc_auc", n_jobs = -1).mean()

            return({"loss" : -score, "status" : STATUS_OK})

        except Exception as e:

            return({"status" : STATUS_FAIL, "exception" : str(e)})

    trials = Trials()

    best_params = fmin(objective, space, algo = tpe.suggest, 
                       max_evals = num_eval, trials = trials)

    return(best_params)

In [329]:
bp = optimize(hyperopt_space, train1, train_label, 100)

100%|██████████| 100/100 [1:00:45<00:00, 36.45s/it, best loss: -0.595608779288199]


In [330]:
bp

{'colsample_by_tree': 0.6404559935062425,
 'learning_rate': 0.018386371949234568,
 'max_depth': 2.0,
 'min_child_weight': 4.0,
 'n_estimators': 100.0,
 'reg_alpha': 0.22603233105740528,
 'reg_lambda': 0.6421403594405247}

In [331]:
bp = optimize(hyperopt_space, train2, train_label, 50)

100%|██████████| 50/50 [2:33:34<00:00, 184.28s/it, best loss: -0.5950995174290206]  


In [333]:
bp

{'colsample_by_tree': 0.6063928129639939,
 'learning_rate': 0.012669428924546472,
 'max_depth': 1.0,
 'min_child_weight': 8.0,
 'n_estimators': 200.0,
 'reg_alpha': 0.15989146089118428,
 'reg_lambda': 0.9690772723763873}

In [204]:
def weight_road_wins(x, weight_road = 1., weight_home = 0.10):
    
    if x == 1:
        
        return(weight_home)
    
    else:
        
        return(weight_road)
    
train_weights2 = train_label.apply(lambda x: weight_road_wins(x))

In [205]:
features = list(train1.columns)

dtrain = DMatrix(train1.values, label = train_label.values, weight = train_weights2.values,
                feature_names = features)

def decay_fn(boosting_round, num_boosting_round):
    
    return( (num_boosting_round / (boosting_round+1)) / 500)

boosting_params = {
    
    "max_depth" : 2,
    
    "eta" : 0.018,
    
    "min_child_weight" : 4,
    
    "colsample_bytree" : 0.6,
    
    "lambda" : 0.64,
    
    "alpha" : 0.22,
    
    "tree_method" : "hist",
    
    "grow_policy" : "lossguide",
    
    "max_bin" : 1024

}

train_params = {
    
    "params" : boosting_params,
    
    "dtrain" : dtrain,
    
    "num_boost_round" : 500,
    
    "nfold" : 5,
    
    "metrics" : ["auc"],
    
    "as_pandas" : True,
    
    "seed" : 42,
}

In [206]:
_eval = Xcv(**train_params)

score = _eval["test-auc-mean"].max()

score, _eval["test-auc-mean"].idxmax()

(0.5970732, 107)

### Build base model to use throughout 2019 season.

In [207]:
base_params = {
    
    "params" : boosting_params,
    
    "dtrain" : dtrain,
    
    "num_boost_round" : 100,
}

xgb_base = Xtrain(**base_params)

In [208]:
@dataclass
class SeasonPipeline():
    data: pd.core.frame.DataFrame
    label: str = "home_win"
    model: object = XGBClassifier()
    weight_dmatrix: bool = False
    train_weights: pd.core.frame.Series = None
    load_model: bool = False
    xgb_model: object = None
    num_boost_round: int = 10
    decay_lr: bool = False
    decay_fn: any = None
    booster_params: dict = field(default_factory = dict)
    train_params: dict = field(default_factory = dict)
    scale_features: bool = False
    scale_type: str = "Standard"
    train_indices: list = field(default_factory = list)
    test_indices: list = field(default_factory = list)
    base: int = 50
    retrain_level: str = "daily"
    time_stack: list = field(default_factory = list)
    index_stack: list = field(default_factory = list)
    preds: list = field(default_factory = list)
    
    def generate_lines(self):
        
        self.simulate_season()
        
        d, p = [], []
        
        for time_unit in self.preds:
            
            for j,k in time_unit:
                
                d.append(j)
                
                p.append(k)
                
        game_day = pd.Series(d).rename("time_unit")
        
        model_preds = pd.Series(p).rename("model_prob")
        
        model_outcome = model_preds.apply(lambda x: self.generate_flat_prediction(x)).rename("flat_outcomes")
        
        model_payouts = model_preds.apply(lambda x: 1. / x).rename("model_payouts")
        
        model_df = pd.concat([game_day, model_preds, model_outcome, model_payouts], axis = 1)
        
        return(model_df)
        
    def simulate_season(self):
        
        self.get_chunks()
        
        train, train_label = self.data.iloc[self.train_indices].drop(self.label, axis = 1),\
        self.data.iloc[self.train_indices][self.label]
        
        self.test_indices = self.index_stack.pop()
        
        test, test_label = self.data.iloc[self.test_indices].drop(self.label, axis = 1),\
        self.data.iloc[self.test_indices][self.label]
        
        if self.scale_features:
            
            train, test = self.scale_features(train, test, method = self.scale_type)
            
        if self.weight_dmatrix:
            
            train_weights = self.train_weights[self.train_indices]
            
            test_weights = self.train_weights[self.test_indices]
            
            preds = self.fit_predict(train, train_label, test, test_label,
                                    train_weights = train_weights, test_weights = test_weights)
            
        else:
            
            preds = self.fit_predict(train, train_label, test, test_label)
            
        time = [self.time_stack.pop()] * len(preds)
            
        self.preds.append(zip(time, preds))
        
        while self.index_stack:
        
            indices = self.index_stack.pop()

            self.train_indices += indices

            self.test_indices = indices

            train, train_label = self.data.iloc[self.train_indices].drop(self.label, axis = 1),\
            self.data.iloc[self.train_indices][self.label]

            test, test_label = self.data.iloc[self.test_indices].drop(self.label, axis = 1),\
            self.data.iloc[self.test_indices][self.label]
            
            if self.scale_features:
            
                train, test = self.scale_features(train, test, method = self.scale_type)
            
            if self.weight_dmatrix:
            
                train_weights = self.train_weights[self.train_indices]

                test_weights = self.train_weights[self.test_indices]

                preds = self.fit_predict(train, train_label, test, test_label,
                                        train_weights = train_weights, test_weights = test_weights)
            
            else:

                preds = self.fit_predict(train, train_label, test, test_label)
                
            time = [self.time_stack.pop()] * len(preds)
            
            self.preds.append(zip(time, preds))
    
    def fit_predict(self, train, train_label, test, test_label, 
                    train_weights = None, test_weights = None):
        
        if self.weight_dmatrix:
            
            dtrain = DMatrix(train.values, label = train_label.values, weight = train_weights.values)
            
            dtest = DMatrix(test.values, label = test_label.values, weight = test_weights.values)
            
        else:
            
            dtrain = DMatrix(train.values, label = train_label.values)
            
            dtest = DMatrix(test.values, label = test_label.values)
                 
        if self.booster_params:

            self.train_params["params"] = self.booster_params

        else:

            self.train_params["params"] = {}

        self.train_params["dtrain"] = dtrain

        if self.decay_lr:
            
            self.train_params["callbacks"] = [reset_learning_rate(self.decay_fn)]
            
        if self.load_model:
            
            self.train_params["xgb_model"] = self.xgb_model
            
            self.train_params["num_boost_round"] = self.num_boost_round
            
        self.xgb_model = Xtrain(**self.train_params)
        
        xgb_preds = self.xgb_model.predict(dtest)
        
        return(xgb_preds)
    
    def get_chunks(self):
        
        if "day_num" not in self.data.columns and "date" in self.data.columns:
            
            d = {}

            counter = 1

            for day in self.data.date.unique():

                d[day] = counter

                counter += 1

            self.data["day_num"] = self.data.date.map(d)
            
            del(d)
        
        self.data["week"] = pd.DatetimeIndex(self.data.date).week
        
        self.data["month"] = pd.DatetimeIndex(self.data.date).month
        
        self.data = self.data.sort_values(by = ["day_num"]).reset_index(drop = True).drop("date", axis = 1)
        
        if self.retrain_level == "daily":
        
            self.train_indices = list(self.data[self.data.day_num <= self.base].index)

            self.time_stack = list(reversed(list(self.data[self.data.day_num > self.base].day_num.unique())))

            for day in self.time_stack:

                self.index_stack.append(list(self.data[self.data.day_num == day].index))

        elif self.retrain_level == "weekly":
            
            base_weeks = list(self.data.week.unique())[:self.base]
            
            self.train_indices = list(self.data[self.data.week.between(base_weeks[0], 
                                                                       base_weeks[-1])].index)
            
            self.time_stack = list(reversed(list(self.data[self.data.week > base_weeks[-1]].week.unique())))
            
            for week in self.time_stack:
                
                self.index_stack.append(list(self.data[self.data.week == week].index))
            
        elif self.retrain_level == "monthly":
            
            base_months = list(self.data.month.unique())[:self.base]
            
            self.train_indices = list(self.data[self.data.month.between(base_months[0], 
                                                                       base_months[-1])].index)
            
            self.time_stack = list(reversed(list(self.data[self.data.month > base_months[-1]].month.unique())))
            
            for month in self.time_stack:
                
                self.index_stack.append(list(self.data[self.data.month == month].index))
                
        self.data.drop(columns = ["day_num", "week", "month"])
            
        assert len(self.time_stack) == len(self.index_stack)
        
    @staticmethod
    def scale_features(train, test, method, scale_binary = False):
        
        def get_binary(df: pd.core.frame.DataFrame):
            
            binary_features = []

            for col in df.columns:

                distinct_vals = df[col].unique()

                if len(distinct_vals) <= 2:

                    binary_features.append(col)

            return(binary_features)
        
        if not scale_binary:

            binary_features = get_binary(train)

            to_scale_train = train.drop(columns = binary_features)
            
            to_scale_test = test.drop(columns = binary_features)

        else:

            to_scale_train = train
            
            to_scale_test = test

        scale_cols = list(to_scale_train.columns)

        if method == "Standard":

            scaler = StandardScaler()

        elif method == "GaussRank":

            scaler = GaussRankScaler()

        elif method == "Robust":

            scaler = RobustScaler()

        elif method == "Quantile":

            scaler = QuantileTransformer()

        elif method == "Power":

            scaler = PowerTransformer()

        else:

            raise ValueError("Did not pass supported scaler.")

        scaler.fit(to_scale_train.values)

        npXtrain = scaler.transform(to_scale_train.values)
        
        npXtest = scaler.transform(to_scale_test.values)

        scaledXtrain = pd.DataFrame(npXtrain)
        
        scaledXtest = pd.DataFrame(npXtest)

        scaledXtrain.columns = scale_cols
        
        scaledXtest.columns = scale_cols

        if binary_features:

            scaledXtrain = pd.concat([scaledXtrain, train[binary_features]], axis = 1)
            
            scaledXtest = pd.concat([scaledXtest, test[binary_features]], axis = 1)

        else:
            
            return(scaledXtrain, scaledXtest)
        
    @staticmethod
    def generate_flat_prediction(x):
        
        if x > 0.5:
            
            return(1)
        
        else:
            
            return(0)

### TRAIN 2019 SEASON

In [209]:
cols = list(train1.columns) + ["home_win", "date"]

all_2019 = df[df.season == 2019][cols].reset_index(drop = True)

train_weights1 = df[df.season == 2019].home_payout.reset_index(drop = True) - 1 #weight individual instances

In [217]:
def weight_underdogs(x):
    
    return(x**2)

train_weights3 = train_weights1.apply(lambda x: weight_underdogs(x))

In [210]:
def weight_road_wins(x, weight_road = 1., weight_home = 0.10):
    
    if x == 1:
        
        return(weight_home)
    
    else:
        
        return(weight_road)
    
train_weights2 = all_2019.home_win.apply(lambda x: weight_road_wins(x))

In [218]:
def decay_fn(boosting_round, num_boosting_round):
    
    return( (num_boosting_round / (boosting_round + 1)) / 200)

SP_params = {
    
    "data" : all_2019,
    
    "weight_dmatrix" : True,
    
    "train_weights" : train_weights3,
    
    "load_model" : True,
    
    "xgb_model" : xgb_base,
    
    "num_boost_round" : 500,
    
    "retrain_level" : "monthly",
    
    "base" : 1,
    
    "scale_features" : False,
    
    "booster_params" : {

        "max_depth" : 2,

        "eta" : 0.018,

        "min_child_weight" : 4,

        "colsample_bytree" : 0.6,

        "lambda" : 0.64,

        "alpha" : 0.22,

        "tree_method" : "hist",

        "grow_policy" : "lossguide",

        "max_bin" : 1024

    }

}

sp = SeasonPipeline(**SP_params)

In [219]:
model_preds = sp.generate_lines()

In [220]:
model_preds

,time_unit,model_prob,flat_outcomes,model_payouts
0,4,0.793281,1,1.260587
1,4,0.524683,1,1.905912
2,4,0.406848,0,2.457920
3,4,0.660136,1,1.514839
4,4,0.771774,1,1.295716
...,...,...,...,...
2327,9,0.584646,1,1.710437
2328,9,0.738931,1,1.353307
2329,9,0.769879,1,1.298905
2330,9,0.509042,1,1.964475


In [221]:
model_preds.flat_outcomes.value_counts()

1    1563
0     769
Name: flat_outcomes, dtype: int64

In [222]:
all_2019["month"] = pd.DatetimeIndex(all_2019.date).month

vals = all_2019[all_2019.month > 3][["date", "home_win", "home_payout", "road_payout"]].reset_index(drop = True)

In [225]:
final = pd.concat([model_preds, vals], axis = 1)

In [226]:
final.head()

,time_unit,model_prob,flat_outcomes,model_payouts,date,home_win,home_payout,road_payout
0,4,0.793281,1,1.260587,2019-04-01,0,1.487805,2.85
1,4,0.524683,1,1.905912,2019-04-01,0,1.606061,2.55
2,4,0.406848,0,2.457920,2019-04-01,1,1.625000,2.50
3,4,0.660136,1,1.514839,2019-04-01,1,1.476190,2.90
4,4,0.771774,1,1.295716,2019-04-01,0,1.568182,2.63


In [227]:
home_dogs = final[(final.model_payouts < 2) & (final.home_payout > 2)]

In [229]:
home_dogs.home_win.value_counts()

0    312
1    205
Name: home_win, dtype: int64